In [121]:
from aeon.datasets import load_classification
X, y, meta_data = load_classification("GunPoint")
print(" Shape of X = ", X.shape)
print(" Meta data = ", meta_data)

 Shape of X =  (200, 1, 150)
 Meta data =  {'problemname': 'gunpoint', 'timestamps': False, 'missing': False, 'univariate': True, 'equallength': True, 'classlabel': True, 'targetlabel': False, 'class_values': ['1', '2']}


In [122]:
X.shape

(200, 1, 150)

In [123]:
import pandas as pd
df = pd.DataFrame(data=X[:, 0, :])
df['label'] = y
df.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,label
0,-0.647885,-0.641992,-0.638186,-0.638259,-0.638345,-0.638697,-0.643049,-0.643768,-0.645050,-0.647118,...,-0.639716,-0.639735,-0.640184,-0.639235,-0.639395,-0.640231,-0.640429,-0.638666,-0.638657,2
1,-0.644427,-0.645401,-0.647055,-0.647492,-0.646910,-0.643884,-0.639731,-0.638094,-0.635297,-0.635384,...,-0.641426,-0.639267,-0.637797,-0.637680,-0.635260,-0.635490,-0.634934,-0.634497,-0.631596,2
2,-0.778353,-0.778279,-0.777151,-0.777684,-0.775900,-0.772421,-0.765464,-0.762275,-0.763752,-0.765356,...,-0.718712,-0.713534,-0.710021,-0.704126,-0.703263,-0.703393,-0.704196,-0.707605,-0.707120,1
3,-0.750060,-0.748103,-0.746164,-0.745926,-0.743767,-0.743805,-0.745213,-0.745082,-0.745727,-0.745815,...,-0.724661,-0.729229,-0.728940,-0.727834,-0.728244,-0.726453,-0.725517,-0.725191,-0.724679,1
4,-0.599539,-0.597422,-0.599269,-0.598259,-0.597582,-0.591303,-0.589020,-0.587533,-0.585462,-0.583847,...,-0.643885,-0.645742,-0.646458,-0.646464,-0.645585,-0.642412,-0.643337,-0.636803,-0.631716,2


In [124]:
df.to_csv('gunpoint_raw.csv', index=False)

In [125]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=0)

In [126]:
X_train.shape

(134, 1, 150)

In [127]:
from sktime.transformations.panel.rocket import Rocket
from sktime.datasets import load_unit_test
import multiprocessing

trf = Rocket(num_kernels=10, n_jobs=multiprocessing.cpu_count()-1) 
trf.fit(X_train) 
X_train_transform = trf.transform(X_train) 
X_test_transform = trf.transform(X_test) 

In [128]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train_transform, y_train)
clf.score(X_test_transform, y_test)

0.8636363636363636

In [129]:
y_test[0:10]

array(['1', '1', '2', '1', '1', '1', '2', '2', '1', '1'], dtype='<U1')

In [130]:
truth_class_one = y_test.astype(int)
truth_class_one[truth_class_one != 1] = 0
truth_class_one[0:10]

array([1, 1, 0, 1, 1, 1, 0, 0, 1, 1])

In [131]:
y_pred = clf.predict(X_test_transform)
y_pred[0:10]

array(['1', '1', '2', '2', '1', '1', '1', '2', '1', '1'], dtype='<U1')

In [132]:
y_pred_proba = clf.predict_proba(X_test_transform)
y_pred_proba[0:10]

array([[0.9202544 , 0.0797456 ],
       [0.92763466, 0.07236534],
       [0.09537825, 0.90462175],
       [0.26502188, 0.73497812],
       [0.9336416 , 0.0663584 ],
       [0.94985878, 0.05014122],
       [0.51431854, 0.48568146],
       [0.32810629, 0.67189371],
       [0.87471776, 0.12528224],
       [0.86671917, 0.13328083]])

In [133]:
proba_class_one = y_pred_proba[:, 0]
proba_class_one[0:10]

array([0.9202544 , 0.92763466, 0.09537825, 0.26502188, 0.9336416 ,
       0.94985878, 0.51431854, 0.32810629, 0.87471776, 0.86671917])

In [134]:
model_error = np.absolute(truth_class_one - proba_class_one)
model_error[0:10]

array([0.0797456 , 0.07236534, 0.09537825, 0.73497812, 0.0663584 ,
       0.05014122, 0.51431854, 0.32810629, 0.12528224, 0.13328083])

In [136]:
df_err = pd.DataFrame(data=X_test[:, 0, :])
df_err['truth'] = y_test
df_err['clf_pred'] = y_pred
df_err['clf_prob'] = proba_class_one
df_err['error'] = model_error
df_err.head()

,0,1,2,3,4,5,6,7,8,9,...,144,145,146,147,148,149,truth,clf_pred,clf_prob,error
0,-1.054014,-1.050717,-1.047666,-1.047448,-1.048769,-1.047781,-1.047891,-1.047536,-1.047369,-1.044656,...,-1.071325,-1.068112,-1.068966,-1.069154,-1.066642,-1.064798,1,1,0.920254,0.079746
1,-0.966288,-0.971525,-0.971881,-0.972672,-0.966935,-0.966532,-0.967584,-0.966147,-0.966501,-0.966086,...,-1.009584,-1.011476,-1.010935,-1.008073,-1.009745,-1.006284,1,1,0.927635,0.072365
2,-0.693911,-0.690750,-0.683657,-0.676097,-0.675943,-0.674141,-0.671976,-0.670974,-0.670276,-0.670103,...,-0.591039,-0.590049,-0.588937,-0.586893,-0.586506,-0.589174,2,2,0.095378,0.095378
3,-0.621861,-0.622251,-0.622982,-0.623165,-0.625679,-0.624674,-0.628023,-0.628529,-0.629481,-0.630061,...,-0.649271,-0.650146,-0.649936,-0.650529,-0.650530,-0.649983,1,2,0.265022,0.734978
4,-0.779429,-0.776860,-0.776728,-0.775657,-0.777356,-0.776667,-0.778071,-0.778201,-0.780394,-0.779949,...,-0.766655,-0.756226,-0.739417,-0.732552,-0.727968,-0.726643,1,1,0.933642,0.066358


In [137]:
df_err.to_csv('gunpoint_processed_error.csv', index=False)